<a href="https://colab.research.google.com/github/RAK789266/http-localhost-8888-notebooks-Untitled13.ipynb-kernel_name-python3/blob/main/ab%20testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
files.upload()

Saving website_ab_test.csv to website_ab_test.csv


{'website_ab_test.csv': b'Theme,Click Through Rate,Conversion Rate,Bounce Rate,Scroll_Depth,Age,Location,Session_Duration,Purchases,Added_to_Cart\nLight Theme,0.05491997767058522,0.28236682633201915,0.40508492068993385,72.48945804392022,25,Chennai,1535,No,Yes\nLight Theme,0.11393184955742346,0.03297320750670295,0.7327587606405821,61.85856842837475,19,Pune,303,No,Yes\nDark Theme,0.32335153007955486,0.17876326658857866,0.2965426969467927,45.73737586862724,47,Chennai,563,Yes,Yes\nLight Theme,0.4858357229118433,0.32522487200020134,0.2450014526264947,76.30529767377683,58,Pune,385,Yes,No\nLight Theme,0.034783470224807767,0.1967657699066717,0.7651004861211368,48.92740687261026,25,New Delhi,1437,No,No\nLight Theme,0.17341948665193285,0.26698689934745345,0.7868249137391361,66.12030031438313,65,Chennai,1244,Yes,No\nLight Theme,0.49452224293129665,0.351191119581454,0.40277035054876154,41.03106882678631,31,Kolkata,1723,Yes,No\nLight Theme,0.3956192689883948,0.21004699664649834,0.7911656600846859,6

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats


In [7]:
data = pd.read_csv('website_ab_test.csv')
print(data.head())

         Theme  Click Through Rate  Conversion Rate  Bounce Rate  \
0  Light Theme            0.054920         0.282367     0.405085   
1  Light Theme            0.113932         0.032973     0.732759   
2   Dark Theme            0.323352         0.178763     0.296543   
3  Light Theme            0.485836         0.325225     0.245001   
4  Light Theme            0.034783         0.196766     0.765100   

   Scroll_Depth  Age   Location  Session_Duration Purchases Added_to_Cart  
0     72.489458   25    Chennai              1535        No           Yes  
1     61.858568   19       Pune               303        No           Yes  
2     45.737376   47    Chennai               563       Yes           Yes  
3     76.305298   58       Pune               385       Yes            No  
4     48.927407   25  New Delhi              1437        No            No  


In [8]:
print(data.isnull().sum())

Theme                 0
Click Through Rate    0
Conversion Rate       0
Bounce Rate           0
Scroll_Depth          0
Age                   0
Location              0
Session_Duration      0
Purchases             0
Added_to_Cart         0
dtype: int64


In [9]:
print(data.describe())

       Click Through Rate  Conversion Rate  Bounce Rate  Scroll_Depth  \
count         1000.000000      1000.000000  1000.000000   1000.000000   
mean             0.256048         0.253312     0.505758     50.319494   
std              0.139265         0.139092     0.172195     16.895269   
min              0.010767         0.010881     0.200720     20.011738   
25%              0.140794         0.131564     0.353609     35.655167   
50%              0.253715         0.252823     0.514049     51.130712   
75%              0.370674         0.373040     0.648557     64.666258   
max              0.499989         0.498916     0.799658     79.997108   

               Age  Session_Duration  
count  1000.000000       1000.000000  
mean     41.528000        924.999000  
std      14.114334        508.231723  
min      18.000000         38.000000  
25%      29.000000        466.500000  
50%      42.000000        931.000000  
75%      54.000000       1375.250000  
max      65.000000       1797.

Now before moving forward, here’s the detail of all the columns you should know:

Theme: dark or light
Click Through Rate: The proportion of the users who click on links or buttons on the website.

Conversion Rate: The percentage of users who signed up on the platform after visiting for the first time.

Bounce Rate: The percentage of users who leave the website without further interaction after visiting a single page.

Scroll Depth: The depth to which users scroll through the website pages.
Age: The age of the user.

Location: The location of the user.

Session Duration: The duration of the user’s session on the website.

Purchases: Whether the user purchased the book (Yes/No).

Added_to_Cart: Whether the user added books to the cart (Yes/No).

In [10]:
# scatter plot for click through rate and conversion rate

fig = px.scatter(data, x = 'Click Through Rate',
                 y='Conversion Rate', color = 'Theme',
                 title='CTR vs conversion rate', trendline='ols'
                 )
fig.show()


The relationship between the Click Through Rate (CTR) and Conversion Rate is consistent and nearly unchanged, as shown by the scatter plot. It means that as more users click on links or buttons (CTR increases), a similar proportion of them also end up signing up daily (Conversion Rate remains stable). In other words, the percentage of users who take the desired action of signing up remains roughly the same regardless of how many users initially clicked on links or buttons to explore the website.



In [11]:
# now, let's have a look at the histogram of the CTR  of both theme

light_theme_data = data[data['Theme']=='Light Theme']
dark_theme_data = data[data['Theme']=='Dark Theme']

# create grouped bar chart for click Through rate

fig = go.Figure()

fig.add_trace(go.Histogram(x=light_theme_data['Click Through Rate'], name='Light Theme', opacity=0.6))
fig.add_trace(go.Histogram(x=dark_theme_data['Click Through Rate'], name='Dark Theme', opacity=0.6))

fig.update_layout(
    title_text = 'Click through rate by theme',
    xaxis_title_text='Click through rate',
    yaxis_title_text='Frequency',
    barmode='group',
    bargap=0.1,
)

fig.show()

We can see in the above histogram that there’s not much difference between the performance of both themes. Now let’s have a look at the histogram of the conversion rates of both themes:

In [12]:
fig=go.Figure()

fig.add_trace(go.Histogram(x=light_theme_data['Conversion Rate'], name='Light Theme', opacity=0.6))
fig.add_trace(go.Histogram(x=dark_theme_data['Conversion Rate'], name='dark theme', opacity=0.6))

fig.update_layout(
    title_text = 'conversion rate by theme',
    xaxis_title_text='conversion rate',
    yaxis_title_text='Frequency',
    barmode='group',
    bargap=0.1
)

fig.show()


Although there’s not much difference, the conversion rate of the dark theme is slightly better than the light theme. Now let’s have a look at the distribution of the bounce rates of both themes:

In [13]:
fig = go.Figure()
fig.add_trace(go.Box(y=light_theme_data['Bounce Rate'],
                     name='Light Theme'))
fig.add_trace(go.Box(y=dark_theme_data['Bounce Rate'],
                     name='Dark Theme'))

fig.update_layout(
    title_text='Bounce Rate by Theme',
    yaxis_title_text='Bounce Rate',
)

fig.show()

There’s not much difference between the bounce rates of both themes still, the bounce rate of the light theme is slightly lower (which means it’s slightly better). Now let’s have a look at the scroll depth of both themes:

In [14]:
fig = go.Figure()
fig.add_trace(go.Box(y=light_theme_data['Scroll_Depth'],
                     name='Light Theme'))
fig.add_trace(go.Box(y=dark_theme_data['Scroll_Depth'],
                     name='Dark Theme'))

fig.update_layout(
    title_text='Scroll Depth by Theme',
    yaxis_title_text='Scroll Depth',
)

fig.show()

**COMPARISON OF BOTH THEME BASED ON PURCHASE**

In [15]:
# AB TESTING FOR PURCHASES

light_theme_conversion = light_theme_data[light_theme_data['Purchases']=='Yes'].shape[0]
light_theme_total=light_theme_data.shape[0]

dark_theme_conversion = dark_theme_data[dark_theme_data['Purchases']=='Yes'].shape[0]
dark_theme_total=dark_theme_data.shape[0]

conversion_count = [light_theme_conversion, dark_theme_conversion]
sample_size = [light_theme_total, dark_theme_total]

light_theme_conversion_rate = light_theme_conversion / light_theme_total
dark_theme_conversion_rate = dark_theme_conversion / dark_theme_total

# perform two - sample proportion test

zstat, pval = proportions_ztest(conversion_count, sample_size)
print("Light Theme Conversion Rate:", light_theme_conversion_rate)
print("Dark Theme Conversion Rate:", dark_theme_conversion_rate)
print("A/B Testing - z-statistic:", zstat, " p-value:", pval)

Light Theme Conversion Rate: 0.5308641975308642
Dark Theme Conversion Rate: 0.5038910505836576
A/B Testing - z-statistic: 0.8531246206222649  p-value: 0.39359019934127804


In the comparison of conversion rates based on purchases from both themes, we conducted an A/B test to determine if there is a statistically significant difference in the conversion rates between the two themes. The results of the A/B test are as follows:


z-statistic: 0.8531
p-value: 0.3936
The z-statistic measures the difference between the conversion rates of the two themes in terms of standard deviations. In this case, the z-statistic is approximately 0.8531. The positive z-statistic value indicates that the conversion rate of the Light Theme is slightly higher than the conversion rate of the Dark Theme.

The p-value represents the probability of observing the observed difference in conversion rates or a more extreme difference if the null hypothesis is true. The null hypothesis assumes that there is no statistically significant difference in conversion rates between the two themes. In this case, the p-value is approximately 0.3936.

Since the p-value is greater than the typical significance level of 0.05 (commonly used in A/B testing), we do not have enough evidence to reject the null hypothesis. It means that the observed difference in conversion rates between the two themes is not statistically significant. The results suggest that any observed difference in the number of purchases could be due to random variation rather than a true difference caused by the themes. In simpler terms, based on the current data and statistical analysis, we cannot confidently say that one theme performs significantly better than the other in terms of purchases.

In [24]:
# error encounter
light_theme_duration = light_theme_data['Session_Duration'].mean()
dark_theme_duration=dark_theme_data['Session_Duration'].mean()

tstat, pval = stats.ttest_ind(light_theme_duration, dark_theme_duration)
print("A/B Testing for Session Duration - t-statistic:", tstat, " p-value:", pval)

A/B Testing for Session Duration - t-statistic: nan  p-value: nan


/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:6951: RuntimeWarning:

invalid value encountered in scalar divide



In [19]:
light_theme_session_duration = light_theme_data['Session_Duration']
dark_theme_session_duration = dark_theme_data['Session_Duration']

# Calculate the average session duration for both themes
light_theme_avg_duration = light_theme_session_duration.mean()
dark_theme_avg_duration = dark_theme_session_duration.mean()

# Print the average session duration for both themes
print("Light Theme Average Session Duration:", light_theme_avg_duration)
print("Dark Theme Average Session Duration:", dark_theme_avg_duration)

# Perform two-sample t-test for session duration
tstat, pval = stats.ttest_ind(light_theme_session_duration, dark_theme_session_duration)
print("A/B Testing for Session Duration - t-statistic:", tstat, " p-value:", pval)

Light Theme Average Session Duration: 930.8333333333334
Dark Theme Average Session Duration: 919.4824902723735
A/B Testing for Session Duration - t-statistic: 0.3528382474155483  p-value: 0.7242842138292167


In [22]:
print(light_theme_avg_duration)

930.8333333333334


In [23]:
print(light_theme_session_duration)

0      1535
1       303
3       385
4      1437
5      1244
       ... 
989     651
991    1025
994     322
997    1237
998     893
Name: Session_Duration, Length: 486, dtype: int64
